<div dir="rtl">

# ۱ – آماده‌سازی داده (Persian News Dataset)

در این نوت‌بوک، دیتاست خبری فارسی را از پوشه‌ی `data/raw` می‌خوانید، یک زیرمجموعه‌ی کوچک و قابل‌مدیریت از آن می‌سازید، داده‌ها را کمی تمیز می‌کنید و نتیجه را در مسیر `data/processed/news_subset.csv` ذخیره می‌کنید.

        https://www.kaggle.com/datasets/amirzenoozi/persian-news-dataset/

</div>


In [ ]:
import re
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm



In [9]:
raw_path = Path("../data/raw/archive_v2.csv")
subset_path = Path("../data/processed/news_subset.csv")

In [10]:
df = pd.read_csv(raw_path)
df.head()

,id,title,short_link,service,subgroup,abstract,body,tags,published_datetime,agency_name
0,1,پیروزی قاطع مس سونگون /نارنجی‌پوشان 6 تایی کردند,http://fna.ir/f1v1mt,استانها,آذربایجان شرقی,تیم مس سونگون برابر مهمان خود به برتری پرگل دس...,به گزارش خبرگزاری فارس از تبریز، در هفته چهارم...,"لیگ برتر, مس, تبریز, اصفهان, مس سونگون, ورزقان...",2021-01-01 11:53:52,FarsNews
1,2,پخت و توزیع آش نذری به‌مناسبت سالگرد شهادت سرد...,http://fna.ir/f1v1mk,استانها,مازندران,پخت آش نذری در مرکز مازندران توسط یکی از عشاق ...,خبرگزاری فارس مازندران ـ نذری پزون یکی از سنت...,"مازندران, حاج قاسم, شهید",2021-01-01 11:51:50,FarsNews
2,3,ترامپ از تجمع بزرگ روز چهارشنبه طرفدارانش در و...,http://fna.ir/f1v1mv,بین الملل,آمریکا، اروپا,رئیس جمهور آمریکا در پیامی در توییتر از تجمع ب...,به گزارش گروه بین‌الملل خبرگزاری فارس، «دونالد...,"واشنگتن, توییتر, آمریکا, مایک پنس",2021-01-01 11:51:41,FarsNews
3,4,پیام تسلیت نماینده ولی‌فقیه در قرارگاه ثارالله...,http://fna.ir/f1v1mh,فرهنگ,حج و زیارت و وقف,نماینده ولی فقیه در قرارگاه ثارالله در پی پیام...,به گزارش خبرگزاری فارس، آیت الله محمدتقی مصباح...,"علی شیرازی, نماینده سابق ولی فقیه در نیروی قدس...",2021-01-01 11:50:42,FarsNews
4,5,بایرن با موسیالا قرارداد حرفه ای منعقد می کند,http://fna.ir/f1v1mq,ورزشی,فوتبال جهان,باشگاه فوتبال بایرن مونیخ با پدیده کم سن و سال...,به گزارش خبرگزاری فارس، «جمال موسیالا» در بازی...,"بایرن مونیخ, فوتبال, بایرن, بوندس‌لیگا",2021-01-01 11:50:00,FarsNews


In [11]:
MINIMUM_CHAR = 1000

df = df.dropna(subset=["id", "title", "body"]).copy()

df["title"] = df["title"].astype(str).str.strip()
df["body"]  = df["body"].astype(str).str.strip()

df = df[(df["title"] != "") & (df["body"] != "")].copy()

df = df[df["body"].str.len() >= MINIMUM_CHAR].reset_index(drop=True)

print("Rows after strict cleaning:", len(df))
df.head()


Rows after strict cleaning: 175796


,id,title,short_link,service,subgroup,abstract,body,tags,published_datetime,agency_name
0,2,پخت و توزیع آش نذری به‌مناسبت سالگرد شهادت سرد...,http://fna.ir/f1v1mk,استانها,مازندران,پخت آش نذری در مرکز مازندران توسط یکی از عشاق ...,خبرگزاری فارس مازندران ـ نذری پزون یکی از سنت...,"مازندران, حاج قاسم, شهید",2021-01-01 11:51:50,FarsNews
1,3,ترامپ از تجمع بزرگ روز چهارشنبه طرفدارانش در و...,http://fna.ir/f1v1mv,بین الملل,آمریکا، اروپا,رئیس جمهور آمریکا در پیامی در توییتر از تجمع ب...,به گزارش گروه بین‌الملل خبرگزاری فارس، «دونالد...,"واشنگتن, توییتر, آمریکا, مایک پنس",2021-01-01 11:51:41,FarsNews
2,7,دانشگاهیان در پیام‌های جداگانه‌ای رحلت آیت‌الل...,http://fna.ir/f1v1mb,سیاسی,تشکل های دانشگاهی,دانشگاهیان و تشکل‌های دانشگاهی دانشگاه‌های مخت...,به گزارش خبرنگار تشکل‌های دانشگاهی خبرگزاری فا...,"دانشگاه, مصباح یزدی, دانشجویان, شهید",2021-01-01 11:35:02,FarsNews
3,8,شمارش معکوس 95ساعته برای کشف خودروی سرقتی در م...,http://fna.ir/f1v1m3,استانها,خراسان رضوی,دربسته خبری ۳ خبر با عناوینی همچون شمارش معکوس...,به گزارش خبرگزاری فارس از مشهد به نقل از پایگا...,"فرمانده انتظامی مشهد, عباس صارمی ساداتی, رئیس ...",2021-01-01 11:34:54,FarsNews
4,9,آیت‌الله مصباح یزدی پیشگام در تحول حوزه بود/ ت...,http://fna.ir/f1v1me,استانها,قم,مدیر حوزه‌های علمیه با تسلیت ارتحال آیت‌الله م...,به گزارش خبرگزاری فارس از قم، در پیام آیت الله...,"مصباح یزدی, شهید, امام خمینی, حوزه‌های علمیه",2021-01-01 11:32:31,FarsNews


In [12]:
N = 10000
per_service = 2000

services = df["service"].dropna().value_counts().head(5).index
df_filt = df[df["service"].isin(services)].copy()


counts = df_filt["service"].value_counts()
print("Counts per service:\n", counts)

take_map = counts.to_dict()

parts = (
    df_filt.groupby("service", group_keys=False)
    .apply(lambda g: g.sample(n=min(per_service, take_map[g.name]), random_state=42))
)

remaining = N - len(parts)
if remaining > 0:
    remaining_pool = df_filt.drop(parts.index)
    extra = remaining_pool.sample(n=remaining, random_state=42)
    df_subset = pd.concat([parts, extra], ignore_index=True)
else:
    df_subset = parts.reset_index(drop=True)

print("Subset size:", len(df_subset))
df_subset["service"].value_counts()



Counts per service:
 service
استانها      85068
بین الملل    20934
جامعه         9193
ورزشی         8936
سیاسی         8751
Name: count, dtype: int64
Subset size: 10000


C:\Users\Asus\AppData\Local\Temp\ipykernel_3236\258555644.py:15: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: g.sample(n=min(per_service, take_map[g.name]), random_state=42))


service
استانها      2000
بین الملل    2000
جامعه        2000
سیاسی        2000
ورزشی        2000
Name: count, dtype: int64

In [13]:
def normalize_fa_basic(text: str) -> str:
    if pd.isna(text):
        return ""
    text = str(text)

    text = text.replace("ي", "ی").replace("ك", "ک")
    text = text.replace("ة", "ه").replace("ؤ", "و").replace("إ", "ا").replace("أ", "ا")

    text = text.replace("\u200c", " ")
    text = re.sub(r"\s+", " ", text).strip()

    return text

df_subset["title"] = df_subset["title"].fillna("").apply(normalize_fa_basic)
df_subset["body"] = df_subset["body"].fillna("").apply(normalize_fa_basic)
df_subset["abstract"] = df_subset["abstract"].fillna("").apply(normalize_fa_basic)

df_subset.head()


,id,title,short_link,service,subgroup,abstract,body,tags,published_datetime,agency_name
0,7910,پرداخت 10 میلیارد ریال تسهیلات گردشگری در زنجا...,http://fna.ir/f1x73c,استانها,زنجان,مدیرکل میراث فرهنگی، صنایع دستی و گردشگری استا...,امیر ارجمند امروز در گفت وگو با خبرنگار فارس د...,"میراث فرهنگی, گردشگری, استان زنجان, صنایع دستی...",2021-01-13 06:38:00,FarsNews
1,144943,حضور 3500 نفر در اردوهای هفت اقلیم در استان مرکزی,http://fna.ir/491pa,استانها,مرکزی,فرمانده سپاه روح الله استان مرکزی گفت: از ابتد...,به گزارش خبرگزاری فارس از اراک سردار سرتیپ دوم...,"امام خمینی, استان مرکزی, اردوهای هفت اقلیم, فر...",2021-10-06 04:04:03,FarsNews
2,229417,آموزش سلامت دوران بلوغ در مدارس عشایر اصفهان ب...,http://mehrnews.com/xZV7j,استانها,اصفهان,اصفهان-مشاور امور بانوان مدیرکل آموزش و پرورش ...,به گزارش خبرگزاری مهر، مرجان پناهی از تهیه بست...,"اداره کل آموزش و پرورش اصفهان, مدارس عشایری, ب...",04/12/2023 10:39:00 AM,MehrNews
3,149383,کیفیت فعلی هوای مشهد ناسالم است/ سال جاری برای...,http://fna.ir/4b6gc,استانها,خراسان رضوی,شاخص کیفیت هوای مشهد طی ۲۴ ساعت گذشته بر روی ع...,به گزارش خبرگزاری فارس از مشهد، بر اساس داده ه...,"کیفیت هوای مشهد, آلودگی هوای مشهد, وضعیت هواش...",2021-10-16 08:25:01,FarsNews
4,103531,12 هزار و 920 همسر شهید یا جانباز در زنجان زند...,http://fna.ir/2iolx,استانها,زنجان,مسوول کمیته زنان کنگره ملی تجلیل از 3 هزار و 5...,به گزارش خبرگزاری فارس در زنجان، ماهرخ بلوری ع...,"ماهرخ بلوری, مسؤول کمیته زنان کنگره ملی تجلیل ...",2021-07-14 09:08:32,FarsNews


In [14]:
subset_path.parent.mkdir(parents=True, exist_ok=True) 
df_subset.to_csv(subset_path, index=False, encoding="utf-8-sig")

print("Saved to:", subset_path.resolve())
print("Rows:", len(df_subset), "| Cols:", df_subset.shape[1])


Saved to: D:\Programming\Bootcamp\دوره ها\RAG2\rag-persian-news-lite-main\data\processed\news_subset.csv
Rows: 10000 | Cols: 10
